## Loading Data

In [1]:
import osmnx as ox
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import time

In [2]:
from scipy.spatial import cKDTree

## Traffic Data

In [3]:
df_traffic  = pd.read_csv('2024_traffic.csv')

In [4]:
df_traffic.head(5)

,idTram,data,estatActual,estatPrevist,Tram_Components,Descripció,Longitud,Latitud
0,1,20240101000552,0,0,1,Diagonal (Ronda de Dalt a Doctor Marañón),2.112035,41.384191
1,1,20240101000552,0,0,2,Diagonal (Ronda de Dalt a Doctor Marañón),2.101503,41.381631
2,1,20240101001553,0,0,1,Diagonal (Ronda de Dalt a Doctor Marañón),2.112035,41.384191
3,1,20240101001553,0,0,2,Diagonal (Ronda de Dalt a Doctor Marañón),2.101503,41.381631
4,1,20240101001052,0,0,1,Diagonal (Ronda de Dalt a Doctor Marañón),2.112035,41.384191


In [5]:
df_traffic['date'] = pd.to_datetime(df_traffic['data'], format='%Y%m%d%H%M%S')#.dt.strftime('%Y-%d-%m')

In [6]:
df_traffic.set_index('date', inplace=True)

In [7]:
df_traffic['coord'] = df_traffic['Latitud'].astype('str') + '_' +df_traffic['Longitud'].astype('str')

In [ ]:
df_traffic.head(10)

In [ ]:
df_traffic['Descripció'].value_counts()

***grouping the traffic data by Descripció, Longitud, and Latitud, then resampling the groups on a daily basis and calculating the median for each day.***

In [8]:
df_traffic_daily = df_traffic[['estatActual',	'estatPrevist',	'Tram_Components', 'Longitud',	'Latitud', 'Descripció']] \
                .groupby(['Descripció', 'Longitud', 'Latitud']) \
                .resample('D') \
                .median() \
                .reset_index()

In [9]:
df_traffic_daily.head(10)

,Descripció,Longitud,Latitud,date,estatActual,estatPrevist,Tram_Components
0,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-01,0.0,0.0,5.0
1,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-02,0.0,0.0,5.0
2,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-03,0.0,0.0,5.0
3,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-04,0.0,0.0,5.0
4,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-05,0.0,0.0,5.0
5,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-06,0.0,0.0,5.0
6,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-07,0.0,0.0,5.0
7,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-08,0.0,0.0,5.0
8,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-09,0.0,0.0,5.0
9,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-10,0.0,0.0,5.0


In [ ]:
df_traffic_daily['Descripció'].value_counts()

### Air_quality

In [10]:
df_air_quality  = pd.read_csv('df_air_quality.csv')

In [11]:
df_air_quality.head(5)

,CODI_PROVINCIA,PROVINCIA,CODI_MUNICIPI,MUNICIPI,ESTACIO,CODI_CONTAMINANT,ANY,MES,DIA,H01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,8,Barcelona,19,Barcelona,4,7,2020,1,1,3.0,...,18.0,V,40.0,V,44.0,V,31.0,V,33.0,V
1,8,Barcelona,19,Barcelona,4,7,2020,1,2,13.0,...,18.0,V,10.0,V,3.0,V,57.0,V,NaN,N
2,8,Barcelona,19,Barcelona,4,7,2020,1,3,33.0,...,56.0,V,46.0,V,40.0,V,32.0,V,NaN,N
3,8,Barcelona,19,Barcelona,4,7,2020,1,4,12.0,...,5.0,V,7.0,V,2.0,V,3.0,V,NaN,N
4,8,Barcelona,19,Barcelona,4,7,2020,1,5,1.0,...,73.0,V,53.0,V,33.0,V,29.0,V,25.0,V


In [63]:
df_air_quality.columns

Index(['CODI_PROVINCIA', 'PROVINCIA', 'CODI_MUNICIPI', 'MUNICIPI', 'ESTACIO',
       'CODI_CONTAMINANT', 'ANY', 'MES', 'DIA', 'H01', 'V01', 'H02', 'V02',
       'H03', 'V03', 'H04', 'V04', 'H05', 'V05', 'H06', 'V06', 'H07', 'V07',
       'H08', 'V08', 'H09', 'V09', 'H10', 'V10', 'H11', 'V11', 'H12', 'V12',
       'H13', 'V13', 'H14', 'V14', 'H15', 'V15', 'H16', 'V16', 'H17', 'V17',
       'H18', 'V18', 'H19', 'V19', 'H20', 'V20', 'H21', 'V21', 'H22', 'V22',
       'H23', 'V23', 'H24', 'V24'],
      dtype='object')

In [72]:
df_air_quality[['V01', 'H02', 'V02', \
       'H03', 'V03', 'H04', 'V04', 'H05', 'V05', 'H06', 'V06', 'H07', 'V07', \
       'H08', 'V08', 'H09', 'V09', 'H10', 'V10', 'H11', 'V11', 'H12', 'V12', \
       'H13', 'V13', 'H14', 'V14', 'H15', 'V15', 'H16', 'V16', 'H17', 'V17', \
       'H18', 'V18', 'H19', 'V19']]

,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,...,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19
0,V,3.00,V,2.00,V,1.00,V,2.00,V,6.00,...,8.0,V,5.0,V,2.0,V,1.0,V,1.0,V
1,V,7.00,V,6.00,V,6.00,V,5.00,V,18.00,...,11.0,V,9.0,V,11.0,V,12.0,V,20.0,V
2,V,13.00,V,16.00,V,3.00,V,1.00,V,4.00,...,14.0,V,9.0,V,16.0,V,20.0,V,59.0,V
3,V,11.00,V,10.00,V,6.00,V,4.00,V,5.00,...,17.0,V,25.0,V,24.0,V,14.0,V,10.0,V
4,V,1.00,V,2.00,V,2.00,V,1.00,V,1.00,...,8.0,V,4.0,V,3.0,V,5.0,V,31.0,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89198,V,1.71,V,1.84,V,1.76,V,1.51,V,1.69,...,NaN,N,NaN,N,NaN,N,NaN,N,NaN,N
89199,V,10.00,V,9.00,V,8.00,V,8.00,V,7.00,...,5.0,V,5.0,V,5.0,V,6.0,V,6.0,V
89200,V,7.00,V,7.00,V,8.00,V,7.00,V,6.00,...,5.0,V,6.0,V,6.0,V,6.0,V,6.0,V
89201,V,7.00,V,6.00,V,7.00,V,8.00,V,8.00,...,13.0,V,12.0,V,11.0,V,11.0,V,12.0,V


## construction

In [12]:
df_constructions  = pd.read_csv('df_constructions.csv')

In [13]:
df_constructions.head(5)

,Any,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Seccio_censal,Any_construccio,Nombre
0,2018,1,Ciutat Vella,1,el Raval,1,Abans de 1901,31
1,2018,1,Ciutat Vella,1,el Raval,1,1901 - 1940,10
2,2018,1,Ciutat Vella,1,el Raval,1,1941 - 1950,10
3,2018,1,Ciutat Vella,1,el Raval,1,1951 - 1960,3
4,2018,1,Ciutat Vella,1,el Raval,1,1961 - 1970,4


In [66]:
df_constructions.tail(5)

,Any,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Seccio_censal,Any_construccio,Nombre
59028,2024,10,Sant Martí,71,Provençals del Poblenou,94,2021-2030,11
59029,2024,10,Sant Martí,71,Provençals del Poblenou,100,2021-2030,4
59030,2024,10,Sant Martí,71,Provençals del Poblenou,101,2021-2030,8
59031,2024,10,Sant Martí,72,Sant Martí de Provençals,110,2021-2030,1
59032,2024,10,Sant Martí,73,la Verneda i la Pau,138,2021-2030,9


In [64]:
df_constructions.columns

Index(['Any', 'Codi_districte', 'Nom_districte', 'Codi_barri', 'Nom_barri',
       'Seccio_censal', 'Any_construccio', 'Nombre'],
      dtype='object')

## Weather

In [14]:
df_weather_neighboor  = pd.read_csv('barcelona_weather_20240101_20240430.csv')

In [15]:
df_weather_neighboor.shape

(8833, 43)

In [16]:
df_weather_neighboor.head(5)

,Unnamed: 0,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,icon,stations,source,latitude,longitude,resolvedAddress,address,timezone,tzoffset,severerisk
0,0,2024-01-01,1704063600,13.4,4.8,7.8,13.4,3.0,6.9,3.7,...,partly-cloudy-day,"['LEBL', 'LEGE', 'AR110', '08184099999', 'LELL...",obs,41.415067,2.107482,"41.4150674,2.1074824","41.4150674,2.1074824",Europe/Madrid,1.0,NaN
1,1,2024-01-02,1704150000,14.6,3.2,9.7,14.6,1.8,8.9,1.3,...,partly-cloudy-day,"['LEBL', 'AR110', '08184099999', 'LELL', '0818...",obs,41.415067,2.107482,"41.4150674,2.1074824","41.4150674,2.1074824",Europe/Madrid,1.0,NaN
2,2,2024-01-03,1704236400,17.1,11.9,14.5,17.1,11.9,14.5,7.6,...,partly-cloudy-day,"['LEBL', 'AR110', '08184099999', 'LELL', '0818...",obs,41.415067,2.107482,"41.4150674,2.1074824","41.4150674,2.1074824",Europe/Madrid,1.0,NaN
3,3,2024-01-04,1704322800,15.5,7.2,10.9,15.5,5.2,10.5,8.0,...,partly-cloudy-day,"['LEBL', 'AR110', '08184099999', 'LELL', '0818...",obs,41.415067,2.107482,"41.4150674,2.1074824","41.4150674,2.1074824",Europe/Madrid,1.0,NaN
4,4,2024-01-05,1704409200,13.4,8.3,10.6,13.4,6.6,10.2,7.4,...,rain,"['LEBL', 'AR110', '08184099999', 'LELL', '0818...",obs,41.415067,2.107482,"41.4150674,2.1074824","41.4150674,2.1074824",Europe/Madrid,1.0,NaN


In [17]:
df_weather_neighboor['date'] = pd.to_datetime(df_weather_neighboor['datetime']).dt.strftime('%Y-%m-%d')

## Agenda

In [18]:
try:
    df_agenda = pd.read_csv('opendatabcn_agenda-csv.csv', encoding="utf-16")
    # Process the DataFrame
except UnicodeDecodeError:
    # Retry reading the file with a different encoding
    try:
        df_agenda = pd.read_csv('opendatabcn_agenda-csv.csv', encoding='latin-1')
        # Process the DataFrame
    except Exception as e:
        print(f"Error reading file: {e}")
        # Handle the error gracefully, such as logging the error message or skipping the file

In [19]:
df_agenda.head(5)

,register_id,name,institution_id,institution_name,created,modified,addresses_roadtype_id,addresses_roadtype_name,addresses_road_id,addresses_road_name,...,secondary_filters_fullpath,secondary_filters_tree,secondary_filters_asia_id,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_lat,geo_epgs_4326_lon,estimated_dates,start_date,end_date
0,﻿99400073961,"56è Torneig Internacional de Polo ""Barcelona P...",NaN,NaN,2004-02-19T12:25:07+01:00,2024-04-18T14:41:00.930162+02:00,NaN,NaN,102306.0,Av Doctor Marañón,...,NaN,NaN,NaN,425954.351830,4.581463e+06,41.381285,2.114450,NaN,2024-05-17T03:00:00+02:00,2024-05-26T03:00:00+02:00
1,﻿99400313074,BRESSOL D'ESTIU EBM L'ALIANÇA - ENGRESCA - ASS...,NaN,NaN,2013-03-14T09:19:24+01:00,2024-03-22T09:32:39.538891+01:00,NaN,NaN,186703.0,C Llull,...,NaN,NaN,NaN,433207.508908,4.583498e+06,41.400248,2.200961,NaN,2024-07-15T03:00:00+02:00,2024-08-02T03:00:00+02:00
2,﻿99400313129,"CASAL D'ESTIU LES ACACIES ""Aventures als 4 reg...",NaN,NaN,2013-03-14T14:16:42+01:00,2024-03-22T09:32:39.319191+01:00,NaN,NaN,270901.0,C Pujades,...,NaN,NaN,NaN,433486.852109,4.583931e+06,41.404176,2.204255,NaN,2024-06-25T03:00:00+02:00,2024-07-31T03:00:00+02:00
3,﻿99400270790,Dates de celebració,NaN,NaN,2011-05-17T16:50:26+02:00,2024-04-11T13:34:56.200197+02:00,NaN,NaN,316809.0,Pg Santa Madrona,...,NaN,NaN,NaN,429635.660306,4.580189e+06,41.370143,2.158620,NaN,2024-06-26T03:00:00+02:00,2024-08-04T03:00:00+02:00
4,﻿99400217668,Visites als horts urbans de la Masia Can Mestr...,NaN,NaN,2008-11-21T13:50:04+01:00,2024-02-16T11:24:40.524295+01:00,NaN,NaN,208205.0,C Menorca,...,NaN,NaN,NaN,432996.621503,4.585782e+06,41.420806,2.198186,NaN,2016-09-18T00:00:00+02:00,NaN


In [73]:
df_agenda[['addresses_roadtype_name',
       'addresses_road_id', 'addresses_road_name',
       'addresses_start_street_number', 'addresses_end_street_number']]

,addresses_roadtype_name,addresses_road_id,addresses_road_name,addresses_start_street_number,addresses_end_street_number
0,NaN,102306.0,Av Doctor Marañón,17.0,NaN
1,NaN,186703.0,C Llull,163.0,NaN
2,NaN,270901.0,C Pujades,252.0,NaN
3,NaN,316809.0,Pg Santa Madrona,38.0,NaN
4,NaN,208205.0,C Menorca,27.0,NaN
...,...,...,...,...,...
5018,NaN,8501.0,C Alfons XII,79.0,NaN
5019,NaN,169409.0,G.V. Corts Catalanes,837.0,NaN
5020,NaN,169409.0,Gran Via de les Corts Catalanes,837.0,NaN
5021,NaN,NaN,NaN,NaN,NaN


In [65]:
df_agenda.columns

Index(['register_id', 'name', 'institution_id', 'institution_name', 'created',
       'modified', 'addresses_roadtype_id', 'addresses_roadtype_name',
       'addresses_road_id', 'addresses_road_name',
       'addresses_start_street_number', 'addresses_end_street_number',
       'addresses_neighborhood_id', 'addresses_neighborhood_name',
       'addresses_district_id', 'addresses_district_name',
       'addresses_zip_code', 'addresses_town', 'addresses_main_address',
       'addresses_type', 'values_id', 'values_attribute_id', 'values_category',
       'values_attribute_name', 'values_value', 'values_outstanding',
       'values_description', 'secondary_filters_id', 'secondary_filters_name',
       'secondary_filters_fullpath', 'secondary_filters_tree',
       'secondary_filters_asia_id', 'geo_epgs_25831_x', 'geo_epgs_25831_y',
       'geo_epgs_4326_lat', 'geo_epgs_4326_lon', 'estimated_dates',
       'start_date', 'end_date'],
      dtype='object')

## Parking Data

In [39]:
df_parking = pd.read_csv('merged_df_trams_colors_horaris_tarifes.csv')

In [40]:
df_parking.head(5)

,ID_TRAM,UTM_IX,UTM_FX,UTM_IY,UTM_FY,LATITUD_I,LONGITUD_I,LATITUD_F,LONGITUD_F,ID_TARIFA,...,PARQUING_SOLS_DINS_HORARI,ID_TIPUS_TARIFA,CODI_TARIFA,TIPUS_FRACCIO,TEMPS_MAXIM,TEMPS_MINIM,DESCRIPTION_TARIFA,DESC_CURTA_TARIFA,IMPORT_FRACCIO,IMPORT_MAXIM
0,7,431371.206,431359.176,4582803.652,4582803.655,41.393835,2.179069,41.393834,2.178925,46,...,0.0,1.0,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0
1,8,431322.330,431311.483,4582399.137,4582420.894,41.390188,2.178531,41.390383,2.178398,52,...,0.0,2.0,RES,2.0,20.0,1.0,"0,20 euros/dia Temp.max Est. 20 dies laborables","0,20 E/d <= 20d",0.2,4.0
2,13,431288.758,431273.443,4582305.752,4582322.247,41.389344,2.178140,41.389491,2.177955,46,...,0.0,1.0,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0
3,14,431301.693,431288.550,4582318.450,4582330.947,41.389459,2.178293,41.389571,2.178134,46,...,0.0,1.0,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0
4,19,431498.403,431489.250,4582755.634,4582746.776,41.393414,2.180596,41.393333,2.180488,54,...,0.0,3.0,MA2,1.0,2.0,0.0,"3,00 euros/hora Temp.max Est. 2 hores","3,00 E/h <= 2h",3.0,6.0


In [41]:
df_parking.columns

Index(['ID_TRAM', 'UTM_IX', 'UTM_FX', 'UTM_IY', 'UTM_FY', 'LATITUD_I',
       'LONGITUD_I', 'LATITUD_F', 'LONGITUD_F', 'ID_TARIFA', 'ID_HORARIO',
       'TIPUS_TRAM', 'ADREÇA', 'PLACES', 'pCOLOR', 'RGB', 'COLOR', 'TYPE',
       'CODI_HORARI', 'DESCRIPTION_HORARI', 'DESC_CURTA_HORARI',
       'INCLUS_FESTIUS', 'PARQUING_SOLS_DINS_HORARI', 'ID_TIPUS_TARIFA',
       'CODI_TARIFA', 'TIPUS_FRACCIO', 'TEMPS_MAXIM', 'TEMPS_MINIM',
       'DESCRIPTION_TARIFA', 'DESC_CURTA_TARIFA', 'IMPORT_FRACCIO',
       'IMPORT_MAXIM'],
      dtype='object')

## Barcelona Street

In [ ]:
# Define the place and network type
place_name = "Barcelona, Spain"
network_type = "all"

In [ ]:
# Get the graph for the specified place
G = ox.graph_from_place(place_name, network_type=network_type, simplify=True)

In [ ]:
# Initialize lists to store street names, coordinates, zones, neighborhoods, and districts
streets = []
coordinates = []
zones = []
neighborhoods = []
districts = []
data_list = []
# Iterate over the graph edges to extract data
for u, v, data in G.edges(data=True):
    # Extract street name
    if data:
        data_list.append(data)
        

    street_name = data.get("name", None)
    if street_name:
        streets.append(street_name)
        
        # Extract coordinates (latitude, longitude) for the start and end nodes
        start_node = G.nodes[u]
        end_node = G.nodes[v]
        start_coords = (start_node["y"], start_node["x"])
        end_coords = (end_node["y"], end_node["x"])
        coordinates.append((start_coords, end_coords))
        
        # Extract zone information
        zone_type = data.get("highway", None)
        if zone_type:
            zones.append(zone_type)
        else:
            zones.append("Unknown")
        
        # Extract neighborhood
        neighborhood = data.get('neighborhood', 'Unknown')
        # neighborhood = data.get("addr:neighbourhood", None)
        neighborhoods.append(neighborhood)
        
        # Extract district
        district = data.get('district', 'Unknown')
        # district = data.get("addr:district", None)
        districts.append(district)

In [ ]:
data = {'streets': streets,
        'coordinates': coordinates,
        'zones': zones}

# Create DataFrame
df_street = pd.DataFrame(data)

In [ ]:
df_street.to_csv('barcelona_street.csv')

In [ ]:
df_street.head(5)

In [ ]:
print(df_street['zones'].value_counts())

## Get Coordinate from Address

In [ ]:
from geopy.geocoders import Nominatim

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_geocoder")
def geo_coding(unique_name):
    data = pd.DataFrame()
    for i, name in enumerate(unique_name):
        # Perform geocoding
        location = geolocator.geocode(name)
        if location:
            latitude = location.latitude
            longitude = location.longitude
            data.loc[i, 'geo_latitude'] = latitude
            data.loc[i, 'geo_longitude'] = longitude
            data.loc[i, 'name'] = name
    data.dropna(inplace=True)
    return data

#### Haversine

In [23]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Earth radius in kilometers
    return distance

### Weather-Traffic

In [24]:
df_traffic_daily_copy = df_traffic_daily.copy()

In [25]:
df_traffic_daily_copy['date'] = df_traffic_daily['date'].dt.strftime('%Y-%m-%d')

In [26]:
df_traffic_daily_copy.head(5)

,Descripció,Longitud,Latitud,date,estatActual,estatPrevist,Tram_Components
0,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-01,0.0,0.0,5.0
1,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-02,0.0,0.0,5.0
2,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-03,0.0,0.0,5.0
3,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-04,0.0,0.0,5.0
4,Almogàvers (Badajoz a Marina),2.187079,41.394682,2024-01-05,0.0,0.0,5.0


In [28]:
def create_join_by_date_distance_coordinate(df_weather, df_traffic):
    """
    # Step 1: Find all the unique combinations of dates between the two dataframes
    # Step 2: Loop through each unique date
    # Step 3: Calculate the distance between coordinates
    """
    results = []
    
    
    unique_dates = set(df_weather['date']).intersection(set(df_traffic['date']))
    
    
    for date in unique_dates:
        weather_subset = df_weather[df_weather['date'] == date]
        traffic_subset = df_traffic[df_traffic['date'] == date]
        if weather_subset.empty or traffic_subset.empty:
            continue
        
        
    
        for index_A, row_A in weather_subset.iterrows():
            distances = [haversine(row_A['latitude'], row_A['longitude'], row_B['Latitud'], \
                                   row_B['Longitud']) for index_B, row_B in traffic_subset.iterrows()]
            nearest_index = distances.index(min(distances))
            merged_row = {**row_A, **traffic_subset.iloc[nearest_index]}
            results.append(merged_row)
    return results

    """
    for index_w, row_w in weather_subset.iterrows():
        for index_t, row_t in traffic_subset.iterrows():
            distance = calculate_distance(row_w['latitude'], row_w['longitude'], row_t['Latitud'], row_t['Longitud'])
            # Step 5: Join by the distance
            if distance < 10:  # Example threshold distance
                merged_data = pd.merge(weather_subset, traffic_subset, on='date', how='inner')
                results.append(merged_data)
"""

In [31]:
traffic_weather = create_join_by_date_distance_coordinate(df_weather_neighboor, df_traffic_daily_copy)

In [32]:
df_traffic_weather = pd.DataFrame(traffic_weather)

In [ ]:
df_traffic_weather[['date','latitude', 'longitude', 'Latitud', 'Longitud']].head(5)

In [ ]:
df_traffic_weather.shape

### Weather-Traffic-Parking

In [36]:
def midpoint(lat1, lon1, lat2, lon2):
    return (lat1 + lat2) / 2, (lon1 + lon2) / 2

In [46]:
df_parking.head(5)

,ID_TRAM,UTM_IX,UTM_FX,UTM_IY,UTM_FY,LATITUD_I,LONGITUD_I,LATITUD_F,LONGITUD_F,ID_TARIFA,...,PARQUING_SOLS_DINS_HORARI,ID_TIPUS_TARIFA,CODI_TARIFA,TIPUS_FRACCIO,TEMPS_MAXIM,TEMPS_MINIM,DESCRIPTION_TARIFA,DESC_CURTA_TARIFA,IMPORT_FRACCIO,IMPORT_MAXIM
0,7,431371.206,431359.176,4582803.652,4582803.655,41.393835,2.179069,41.393834,2.178925,46,...,0.0,1.0,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0
1,8,431322.330,431311.483,4582399.137,4582420.894,41.390188,2.178531,41.390383,2.178398,52,...,0.0,2.0,RES,2.0,20.0,1.0,"0,20 euros/dia Temp.max Est. 20 dies laborables","0,20 E/d <= 20d",0.2,4.0
2,13,431288.758,431273.443,4582305.752,4582322.247,41.389344,2.178140,41.389491,2.177955,46,...,0.0,1.0,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0
3,14,431301.693,431288.550,4582318.450,4582330.947,41.389459,2.178293,41.389571,2.178134,46,...,0.0,1.0,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0
4,19,431498.403,431489.250,4582755.634,4582746.776,41.393414,2.180596,41.393333,2.180488,54,...,0.0,3.0,MA2,1.0,2.0,0.0,"3,00 euros/hora Temp.max Est. 2 hores","3,00 E/h <= 2h",3.0,6.0


In [48]:
# Calculate midpoints of parking segments
df_parking['mid_latitude'] = df_parking.apply(
    lambda row: midpoint(row['LATITUD_I'], row['LONGITUD_I'], row['LATITUD_F'], row['LONGITUD_F'])[0], axis=1)
df_parking['mid_longitude'] = df_parking.apply(
    lambda row: midpoint(row['LATITUD_I'], row['LONGITUD_I'], row['LATITUD_F'], row['LONGITUD_F'])[1], axis=1)

In [50]:
df_parking.head(5)

,ID_TRAM,UTM_IX,UTM_FX,UTM_IY,UTM_FY,LATITUD_I,LONGITUD_I,LATITUD_F,LONGITUD_F,ID_TARIFA,...,CODI_TARIFA,TIPUS_FRACCIO,TEMPS_MAXIM,TEMPS_MINIM,DESCRIPTION_TARIFA,DESC_CURTA_TARIFA,IMPORT_FRACCIO,IMPORT_MAXIM,mid_latitude,mid_longitude
0,7,431371.206,431359.176,4582803.652,4582803.655,41.393835,2.179069,41.393834,2.178925,46,...,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0,41.393835,2.178997
1,8,431322.330,431311.483,4582399.137,4582420.894,41.390188,2.178531,41.390383,2.178398,52,...,RES,2.0,20.0,1.0,"0,20 euros/dia Temp.max Est. 20 dies laborables","0,20 E/d <= 20d",0.2,4.0,41.390285,2.178465
2,13,431288.758,431273.443,4582305.752,4582322.247,41.389344,2.178140,41.389491,2.177955,46,...,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0,41.389417,2.178047
3,14,431301.693,431288.550,4582318.450,4582330.947,41.389459,2.178293,41.389571,2.178134,46,...,A,1.0,2.0,0.0,"2,50 euros/hora Temp.max Est. 2 hores","2,50 E/h <= 2h",2.5,5.0,41.389515,2.178214
4,19,431498.403,431489.250,4582755.634,4582746.776,41.393414,2.180596,41.393333,2.180488,54,...,MA2,1.0,2.0,0.0,"3,00 euros/hora Temp.max Est. 2 hores","3,00 E/h <= 2h",3.0,6.0,41.393373,2.180542


In [61]:
def find_nearest_weather(df_parking, weather_df):
    results = []
    for index_A, row_A in weather_df.iterrows():
        distances = [haversine(row_A['latitude'], row_A['longitude'], row_B['mid_latitude'], \
                               row_B['mid_longitude']) for index_B, row_B in df_parking.iterrows()]
        nearest_index = distances.index(min(distances))
        merged_row = {**row_A, **df_parking.iloc[nearest_index]}
        results.append(merged_row)
    return results

In [62]:
parking_weather = find_nearest_weather(df_parking, df_weather_neighboor)

In [68]:
df_parking_weather = pd.DataFrame(parking_weather)

In [69]:
df_parking_weather.head(5)

,Unnamed: 0,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,CODI_TARIFA,TIPUS_FRACCIO,TEMPS_MAXIM,TEMPS_MINIM,DESCRIPTION_TARIFA,DESC_CURTA_TARIFA,IMPORT_FRACCIO,IMPORT_MAXIM,mid_latitude,mid_longitude
0,0,2024-01-01,1704063600,13.4,4.8,7.8,13.4,3.0,6.9,3.7,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408967,2.109554
1,1,2024-01-02,1704150000,14.6,3.2,9.7,14.6,1.8,8.9,1.3,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408967,2.109554
2,2,2024-01-03,1704236400,17.1,11.9,14.5,17.1,11.9,14.5,7.6,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408967,2.109554
3,3,2024-01-04,1704322800,15.5,7.2,10.9,15.5,5.2,10.5,8.0,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408967,2.109554
4,4,2024-01-05,1704409200,13.4,8.3,10.6,13.4,6.6,10.2,7.4,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408967,2.109554


In [71]:
df_parking_weather.tail(5)

,Unnamed: 0,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,CODI_TARIFA,TIPUS_FRACCIO,TEMPS_MAXIM,TEMPS_MINIM,DESCRIPTION_TARIFA,DESC_CURTA_TARIFA,IMPORT_FRACCIO,IMPORT_MAXIM,mid_latitude,mid_longitude
8828,8828,2024-04-26,1714082400,17.7,9.1,14.3,17.7,9.1,14.2,9.6,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408845,2.104516
8829,8829,2024-04-27,1714168800,17.9,14.5,16.1,17.9,14.5,16.1,12.8,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408845,2.104516
8830,8830,2024-04-28,1714255200,16.2,14.4,15.2,16.2,14.4,15.2,12.9,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408845,2.104516
8831,8831,2024-04-29,1714341600,14.2,12.4,13.3,14.2,12.4,13.3,12.0,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408845,2.104516
8832,8832,2024-04-30,1714428000,18.0,13.2,14.9,18.0,13.2,14.9,12.0,...,MB2,1.0,2.0,0.0,"2,75 euros/hora Temp.max Est. 2 hores","2,75 E/h <= 2h",2.75,5.5,41.408845,2.104516


In [74]:
df_parking_weather.shape

(8833, 78)

### Weather-Traffic-Parking

In [75]:
traffic_parking_weather_weather = create_join_by_date_distance_coordinate(df_parking_weather, df_traffic_daily_copy)

In [78]:
df_traffic_parking_weather_weather = pd.DataFrame(traffic_parking_weather_weather)

In [79]:
df_traffic_parking_weather_weather.head(10)

,Unnamed: 0,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,IMPORT_FRACCIO,IMPORT_MAXIM,mid_latitude,mid_longitude,Descripció,Longitud,Latitud,estatActual,estatPrevist,Tram_Components
0,8,2024-01-09,1704754800,11.3,-0.1,4.8,11.3,-4.3,2.7,-0.4,...,2.75,5.5,41.408967,2.109554,Ronda de Dalt (Nus de la Trinitat a Via Júlia),2.117802,41.404822,2.0,2.0,5.0
1,129,2024-01-09,1704754800,11.4,0.6,5.1,11.4,-3.9,2.8,-0.3,...,0.00,0.0,41.400150,2.121250,Pg. Reina Elisenda (Av. Pedralbes a Via Augusta),2.121481,41.399948,1.0,1.0,12.0
2,250,2024-01-09,1704754800,11.5,1.0,5.4,11.5,-3.6,3.1,-0.2,...,2.75,5.5,41.389984,2.112233,Av. Esplugues (Ronda de Dalt a Av. Pedralbes),2.111233,41.391342,1.0,1.0,6.0
3,371,2024-01-09,1704754800,11.5,1.2,5.5,11.5,-3.4,3.1,-0.2,...,2.75,5.5,41.382225,2.126402,Travessera de les Corts (Gran Via Carles III a...,2.126331,41.381148,0.0,0.0,2.0
4,492,2024-01-09,1704754800,11.3,0.3,4.9,11.3,-4.1,2.7,-0.4,...,2.75,5.5,41.405642,2.133303,Muntaner (Pl. Bonanova a Mitre),2.133793,41.406078,1.0,1.0,2.0
5,613,2024-01-09,1704754800,11.4,0.6,5.1,11.4,-3.9,2.8,-0.3,...,2.75,5.5,41.397441,2.130477,Doctor Roux (Mitre a Via Augusta),2.130205,41.397725,0.0,0.0,2.0
6,734,2024-01-09,1704754800,11.5,1.1,5.4,11.5,-3.5,3.1,-0.2,...,2.75,5.5,41.384700,2.133318,Travessera de les Corts (Numància a Carles III),2.130862,41.384185,1.0,1.0,2.0
7,855,2024-01-09,1704754800,11.5,1.3,5.7,11.5,-3.3,3.2,-0.1,...,0.00,0.0,41.375328,2.127332,Creu Coberta - Sants (Gran Via Carles III a Ri...,2.130243,41.375532,1.0,1.0,2.0
8,976,2024-01-09,1704754800,11.2,-0.6,4.4,11.2,-4.5,2.6,-0.6,...,2.25,4.5,41.428625,2.144380,Granja Vella (Pl. de la Clota a Ronda de Dalt),2.143896,41.427407,0.0,0.0,7.0
9,1097,2024-01-09,1704754800,11.2,-0.6,4.4,11.2,-4.6,2.5,-0.6,...,2.75,5.5,41.430161,2.148708,Ronda de Dalt (Av. Jordà a Av. Estatut de Cata...,2.146477,41.433398,2.0,2.0,2.0


In [82]:
df_traffic_parking_weather_weather.to_csv('df_traffic_parking_weather_weather.csv')

### Technical Report on the Logic and Implementation of the Provided Code

#### Overview
The provided code consists of three main components:
1. `haversine` function: Calculates the distance between two geographical points.
2. `create_join_by_date_distance_coordinate` function: Joins two dataframes (traffic and weather data) based on date and geographical proximity.
3. Additional logic: Calculates midpoints of parking segments, finds the nearest weather data to these midpoints, and joins the resulting data with weather data.

Below, we detail the logic and implementation of these components.

### 1. Haversine Function

##### Purpose
The `haversine` function calculates the great-circle distance between two points on the Earth's surface given their latitude and longitude. This distance is the shortest path between two points on a sphere.

##### Implementation
The Haversine formula is used to compute the distance between two geographic coordinates (latitude and longitude).

```python
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Earth radius in kilometers
    return distance
```

- **Conversion to Radians**: Latitude and longitude values are converted from degrees to radians for trigonometric calculations.
- **Difference Calculation**: Compute the differences between the latitudes and longitudes (`dlat` and `dlon`).
- **Haversine Formula**: Apply the Haversine formula to calculate the central angle `c` between the points.
- **Distance Calculation**: Multiply the central angle `c` by the Earth's radius (6371 km) to obtain the distance.

### 2. Create Join by Date and Distance Coordinate Function

##### Purpose
The `create_join_by_date_distance_coordinate` function joins two dataframes (traffic and weather data) based on date and geographical proximity.

##### Implementation

```python
def create_join_by_date_distance_coordinate(df_weather, df_traffic):
    """
    Step 1: Find all the unique combinations of dates between the two dataframes
    Step 2: Loop through each unique date
    Step 3: Calculate the distance between coordinates
    """
    results = []
    
    unique_dates = set(df_weather['date']).intersection(set(df_traffic['date']))
    
    for date in unique_dates:
        weather_subset = df_weather[df_weather['date'] == date]
        traffic_subset = df_traffic[df_traffic['date'] == date]
        if weather_subset.empty or traffic_subset.empty:
            continue
    
        for index_A, row_A in weather_subset.iterrows():
            distances = [haversine(row_A['latitude'], row_A['longitude'], row_B['Latitud'], \
                                   row_B['Longitud']) for index_B, row_B in traffic_subset.iterrows()]
            nearest_index = distances.index(min(distances))
            merged_row = {**row_A, **traffic_subset.iloc[nearest_index]}
            results.append(merged_row)
    return results
```

- **Unique Dates**: Find the intersection of dates between the two dataframes.
- **Loop through Dates**: For each unique date, create subsets of weather and traffic data.
- **Distance Calculation**: Calculate the distance between each pair of weather and traffic data points using the `haversine` function.
- **Join Data**: Merge the closest traffic data point with the weather data point and append the result to the list.

### 3. Additional Logic for Joining Parking Data

##### Purpose
To extend the join operation to include parking data by calculating midpoints of parking segments and finding the nearest weather data points to these midpoints.

##### Implementation

```python
def midpoint(lat1, lon1, lat2, lon2):
    return (lat1 + lat2) / 2, (lon1 + lon2) / 2

# Calculate midpoints of parking segments
df_parking['mid_latitude'] = df_parking.apply(
    lambda row: midpoint(row['LATITUD_I'], row['LONGITUD_I'], row['LATITUD_F'], row['LONGITUD_F'])[0], axis=1)
df_parking['mid_longitude'] = df_parking.apply(
    lambda row: midpoint(row['LATITUD_I'], row['LONGITUD_I'], row['LATITUD_F'], row['LONGITUD_F'])[1], axis=1)

def find_nearest_weather(df_parking, weather_df):
    results = []
    for index_A, row_A in weather_df.iterrows():
        distances = [haversine(row_A['latitude'], row_A['longitude'], row_B['mid_latitude'], \
                               row_B['mid_longitude']) for index_B, row_B in df_parking.iterrows()]
        nearest_index = distances.index(min(distances))
        merged_row = {**row_A, **df_parking.iloc[nearest_index]}
        results.append(merged_row)
    return results

parking_weather = find_nearest_weather(df_parking, df_weather_neighboor)
```

- **Calculate Midpoints**: Use the `midpoint` function to compute the midpoint coordinates of parking segments and add them as new columns in the dataframe.
- **Find Nearest Weather**: For each weather data point, calculate the distance to all parking segment midpoints using the `haversine` function, and find the nearest parking segment.
- **Merge Data**: Combine the weather data point with the closest parking segment data and append the result to the list.

### Combining All Data

Finally, the combined weather and parking data is further joined with the traffic data.

```python
traffic_parking_weather = create_join_by_date_distance_coordinate(parking_weather, df_traffic_daily_copy)
```

- **Join Dataframes**: Use the `create_join_by_date_distance_coordinate` function to merge the combined weather and parking data (`parking_weather`) with the traffic data (`df_traffic_daily_copy`) based on date and proximity of coordinates.

### Conclusion
The provided code effectively calculates distances between geographical points, computes midpoints of parking segments, and performs complex joins of multiple dataframes based on date and proximity. The result is a comprehensive dataset that combines weather, parking, and traffic information, allowing for sophisticated analysis and insights.